In [8]:
import numpy as np
import pandas as pd
import datetime
import seaborn as sns
import plotly.express as px 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
pd.set_option('display.max_columns', 500)

In [34]:
realtor_data_raw = pd.read_csv("https://econdata.s3-us-west-2.amazonaws.com/Reports/Core/RDC_Inventory_Core_Metrics_State_History.csv")

# for some reason the final row was corrupt with a note
realtor_data = realtor_data_raw.drop(realtor_data_raw.tail(1).index)

realtor_data = (realtor_data
    .assign(month_date_yyyymm = (realtor_data['month_date_yyyymm'] + "01").astype('datetime64'))
    .rename(columns = {'month_date_yyyymm':'month'})
    .astype({'state':'category','state_id':'category'})
    .sort_values(by = ['state','month'], ascending = True)
    )



In [4]:
realtor_data.columns

Index(['month', 'state', 'state_id', 'median_listing_price',
       'median_listing_price_mm', 'median_listing_price_yy',
       'active_listing_count', 'active_listing_count_mm',
       'active_listing_count_yy', 'median_days_on_market',
       'median_days_on_market_mm', 'median_days_on_market_yy',
       'new_listing_count', 'new_listing_count_mm', 'new_listing_count_yy',
       'price_increased_count', 'price_increased_count_mm',
       'price_increased_count_yy', 'price_reduced_count',
       'price_reduced_count_mm', 'price_reduced_count_yy',
       'pending_listing_count', 'pending_listing_count_mm',
       'pending_listing_count_yy', 'median_listing_price_per_square_foot',
       'median_listing_price_per_square_foot_mm',
       'median_listing_price_per_square_foot_yy', 'median_square_feet',
       'median_square_feet_mm', 'median_square_feet_yy',
       'average_listing_price', 'average_listing_price_mm',
       'average_listing_price_yy', 'total_listing_count',
       'total

In [5]:
cols = ['month', 'state', 'state_id', 
       'median_listing_price',
       #'median_listing_price_mm', 'median_listing_price_yy',
       'active_listing_count', #'active_listing_count_mm',
       #'active_listing_count_yy', 
       'median_days_on_market',
       #'median_days_on_market_mm', 'median_days_on_market_yy',
       'new_listing_count', 
       #'new_listing_count_mm', 'new_listing_count_yy',
       'price_increased_count', 
       #'price_increased_count_mm','price_increased_count_yy', 
       'price_reduced_count',
       #'price_reduced_count_mm', 'price_reduced_count_yy',
       'pending_listing_count', 
       #'pending_listing_count_mm', 'pending_listing_count_yy',
       #'median_listing_price_per_square_foot',
       #'median_listing_price_per_square_foot_mm','median_listing_price_per_square_foot_yy', 
       'median_square_feet',
       #'median_square_feet_mm', 'median_square_feet_yy',
       'average_listing_price', 
       #'average_listing_price_mm', 'average_listing_price_yy', 
       # 'total_listing_count',
       #'total_listing_count_mm', 'total_listing_count_yy', 'pending_ratio',
       #'pending_ratio_mm', 'pending_ratio_yy', 'quality_flag'
       ]

realtor_data[cols].head()

,month,state,state_id,median_listing_price,active_listing_count,median_days_on_market,new_listing_count,price_increased_count,price_reduced_count,pending_listing_count,median_square_feet,average_listing_price
3783,2016-07-01,Alabama,AL,185448.0,30503.0,90.0,7828.0,364.0,6078.0,1872.0,1974.0,251720.0
3739,2016-08-01,Alabama,AL,185000.0,29933.0,94.0,7182.0,452.0,5864.0,1905.0,1974.0,250583.0
3683,2016-09-01,Alabama,AL,184900.0,29563.0,96.0,7066.0,424.0,5790.0,1736.0,1959.0,248467.0
3621,2016-10-01,Alabama,AL,182900.0,28927.0,100.0,6420.0,520.0,5272.0,1617.0,1950.0,250291.0
3603,2016-11-01,Alabama,AL,180500.0,27931.0,107.0,5680.0,384.0,3946.0,1452.0,1950.0,247254.0


In [35]:
realtor_data = realtor_data.assign(
        month_num = lambda x: x.month.dt.month,
        year = lambda x: x.month.dt.year)

# percent change month over month
realtor_data['pct_ch_med_list_mm'] = realtor_data.median_listing_price.pct_change(periods = 1)*100
realtor_data['pct_ch_active_list_mm'] = realtor_data.active_listing_count.pct_change(periods = 1)*100
realtor_data['pct_ch_med_days_mm'] = realtor_data.median_days_on_market.pct_change(periods = 1)*100
realtor_data['pct_ch_new_list_mm'] = realtor_data.new_listing_count.pct_change(periods = 1)*100
realtor_data['pct_ch_price_inc_mm'] = realtor_data.price_increased_count.pct_change(periods = 1)*100
realtor_data['pct_ch_price_red_mm'] = realtor_data.price_reduced_count.pct_change(periods = 1)*100
realtor_data['pct_ch_pending_mm'] = realtor_data.pending_listing_count.pct_change(periods = 1)*100

# percent change year over year
realtor_data['pct_ch_med_list_yy'] = realtor_data.median_listing_price.pct_change(periods =12)*100
realtor_data['pct_ch_active_list_yy'] = realtor_data.active_listing_count.pct_change(periods =12)*100
realtor_data['pct_ch_med_days_yy'] = realtor_data.median_days_on_market.pct_change(periods =12)*100
realtor_data['pct_ch_new_list_yy'] = realtor_data.new_listing_count.pct_change(periods =12)*100
realtor_data['pct_ch_price_inc_yy'] = realtor_data.price_increased_count.pct_change(periods =12)*100
realtor_data['pct_ch_price_red_yy'] = realtor_data.price_reduced_count.pct_change(periods =12)*100
realtor_data['pct_ch_pending_yy'] = realtor_data.pending_listing_count.pct_change(periods =12)*100

In [43]:
realtor_data[['state','pct_ch_med_list_yy']].tail()

,state,pct_ch_med_list_yy
227,Wyoming,33.542373
190,Wyoming,30.635654
140,Wyoming,35.981453
70,Wyoming,37.257597
15,Wyoming,30.303030


In [51]:
# biggest increases
a = realtor_data.query("month == '2022-09-01'").sort_values(by  = 'pct_ch_med_list_yy', ascending=True).tail(10)
# smallest increase (biggest decreases)False
b = realtor_data.query("month == '2022-09-01'").sort_values(by  = 'pct_ch_med_list_yy', ascending=False).tail(10)


fig = make_subplots(
        rows=1, 
        cols=2,
        subplot_titles=('Largest Increases', 'Smallest Increases'))

fig.add_trace(
    go.Bar(x=a.pct_ch_med_list_yy, y=a.state, orientation= 'h'),
    row=1, col=1)

fig.add_trace(
    go.Bar(x=b.pct_ch_med_list_yy, y=b.state, orientation = 'h'),
    row=1, col=2
)

# update y axes
fig.update_xaxes(title_text="Percent Chance in Median List Prices", row=1, col=1)
fig.update_xaxes(title_text="Percent Chance in Median List Prices", row=1, col=2)

fig.update_layout(title_text="September 2022: States with the largest and smallest gains in list prices year over year")
fig.show()

In [59]:
select_states = ['NE','KA','WY','CA','OR','PA','CO','IL','WI','UT','ID','MI','NY']
select_states

['NE', 'KA', 'WY', 'CA', 'OR', 'PA', 'CO', 'IL', 'WI', 'UT', 'ID', 'MI', 'NY']

In [61]:
fig = px.line(realtor_data.query("state_id in @select_states"),
    x = 'month_num',
    y = 'median_listing_price',
    color = 'year',
    facet_col = 'state',
    facet_col_wrap = 4,
    title = 'Median List Prices of Homes Over Time by State',
    color_discrete_sequence=px.colors.sequential.Inferno
    )
fig.update_yaxes(matches = None)
fig.update_layout(height = 800, width = 1600)
fig.show()

In [97]:
c = realtor_data.query("month in ['2016-09-01','2022-09-01']")[['month','state','median_listing_price']].pivot(index = 'state',columns = 'month',values = 'median_listing_price').reset_index()
c['pct_change'] = (c.iloc[:,2] - c.iloc[:,1])/c.iloc[:,1]*100

a = c.sort_values(by = 'pct_change', ascending = False).head(10)
b = c.sort_values(by = 'pct_change', ascending =True).head(10)

a


month,state,2016-09-01 00:00:00,2022-09-01 00:00:00,pct_change
12,Idaho,258998.0,549900.0,112.318242
26,Montana,309250.0,629950.0,103.702506
32,New York,299950.0,582450.0,94.182364
42,Tennessee,219900.0,425000.0,93.269668
14,Indiana,145000.0,271200.0,87.034483
49,Wisconsin,188450.0,349900.0,85.672592
47,Washington,342500.0,629993.0,83.939562
29,New Hampshire,275000.0,499450.0,81.618182
16,Kansas,156450.0,280595.0,79.351230
44,Utah,325000.0,579500.0,78.307692


In [98]:
b

month,state,2016-09-01 00:00:00,2022-09-01 00:00:00,pct_change
8,District of Columbia,549950.0,621250.0,12.964815
34,North Dakota,239900.0,289900.0,20.842018
11,Hawaii,649963.0,865500.0,33.161426
20,Maryland,299900.0,399950.0,33.361120
13,Illinois,219999.0,299000.0,35.909709
48,West Virginia,159813.0,218575.0,36.769224
18,Louisiana,205428.0,284950.0,38.710400
1,Alaska,285000.0,399700.0,40.245614
5,Colorado,423315.0,600000.0,41.738422
43,Texas,274700.0,389945.0,41.953040


In [96]:
fig = make_subplots(
        rows=1, 
        cols=2,
        subplot_titles=('Largest Increases', 'Smallest Increases'))

fig.add_trace(
    go.Bar(x=a.pct_change, y=a.state, orientation= 'h'),
    row=1, col=1)

fig.add_trace(
    go.Bar(x=b.pct_change, y=b.state, orientation = 'h'),
    row=1, col=2
)

# update y axes
fig.update_xaxes(title_text="Percent Chance in Median List Prices", row=1, col=1)
fig.update_xaxes(title_text="Percent Chance in Median List Prices", row=1, col=2)

fig.update_layout(title_text="States with largest and smallest appreciation since September 2016")
fig.show()

ValueError: 
    Invalid value of type 'builtins.method' received for the 'x' property of bar
        Received value: <bound method NDFrame.pct_change of month          state  2016-09-01 00:00:00  2022-09-01 00:00:00  pct_change
12             Idaho             258998.0             549900.0  112.318242
26           Montana             309250.0             629950.0  103.702506
32          New York             299950.0             582450.0   94.182364
42         Tennessee             219900.0             425000.0   93.269668
14           Indiana             145000.0             271200.0   87.034483
49         Wisconsin             188450.0             349900.0   85.672592
47        Washington             342500.0             629993.0   83.939562
29     New Hampshire             275000.0             499450.0   81.618182
16            Kansas             156450.0             280595.0   79.351230
44              Utah             325000.0             579500.0   78.307692>

    The 'x' propert